![image.png](https://i.imgur.com/a3uAqnb.png)

# Prompt Engineering: Mastering the Art of AI Communication

This notebook demonstrates the fundamentals of **Prompt Engineering** - the practice of designing effective prompts to get better outputs from Large Language Models (LLMs). We'll explore different techniques that can dramatically improve model performance without any training or fine-tuning!

### **📌 The Core Idea: Prompt Engineering**
Prompt engineering is like learning to communicate effectively with an AI. Just as you might phrase a question differently for a child versus a professor, we need to craft our prompts to get the best responses from AI models.

**What we'll cover:**
1. **Basic Prompting**: Simple, direct instructions
2. **Few-shot Learning**: Providing examples to guide the model
3. **Chain-of-Thought (CoT)**: Teaching the model to reason step-by-step
4. **Zero-shot CoT**: Getting reasoning without examples

We'll use high-quality models that work well on 8GB VRAM: **Meta-Llama-3.2-3B-Instruct** for instruction following and **microsoft/Phi-3-mini-4k-instruct** for reasoning tasks - both are significantly more capable than older models!

In [ ]:
# Install required packages (run this first)
# %pip install torch torchvision torchaudio transformers accelerate bitsandbytes

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
import warnings
warnings.filterwarnings('ignore')

# Suppress transformers warnings for clean output
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

print("📚 Prompt Engineering Lab Setup Complete!")
print(f"🔧 Using device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

## 1️⃣ Model Setup: High-Quality Models for 8GB VRAM

For this lab, we'll use two state-of-the-art models optimized for performance:

- **Meta-Llama-3.2-3B-Instruct**: A 3B parameter model with excellent instruction following
- **Microsoft Phi-3-mini-4k-instruct**: A 3.8B parameter model optimized for reasoning

Both models are:
✅ **High Quality**: Much better outputs than older models
✅ **8GB VRAM Compatible**: Using 4-bit quantization
✅ **Fast Inference**: Optimized for efficiency
✅ **Instruction Tuned**: Designed to follow prompts well

These models represent the current state-of-the-art for efficient, high-quality language models!

In [ ]:
def setup_quantization():
    """Setup 4-bit quantization for memory efficiency"""
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

def load_models():
    """Load our high-quality prompt engineering models"""

    quantization_config = setup_quantization() if torch.cuda.is_available() else None

    print("🔄 Loading Llama-3.2-3B-Instruct for instruction following...")
    # Llama 3.2 3B Instruct - excellent for instruction following
    llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
    llama_model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-3.2-3B-Instruct",
        quantization_config=quantization_config,
        device_map="auto" if torch.cuda.is_available() else None,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        trust_remote_code=True,
        attn_implementation="eager"
    )

    print("🔄 Loading Phi-3-mini for reasoning tasks...")
    # Phi-3-mini - use older revision to avoid cache issues
    phi_tokenizer = AutoTokenizer.from_pretrained(
        "microsoft/Phi-3-mini-4k-instruct",
        revision="main"
    )
    phi_model = AutoModelForCausalLM.from_pretrained(
        "microsoft/Phi-3-mini-4k-instruct",
        quantization_config=quantization_config,
        device_map="auto" if torch.cuda.is_available() else None,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        trust_remote_code=True,
        attn_implementation="eager",
        revision="main",
        use_cache=False  # Disable cache to avoid compatibility issues
    )

    # Set padding tokens
    if llama_tokenizer.pad_token is None:
        llama_tokenizer.pad_token = llama_tokenizer.eos_token
    if phi_tokenizer.pad_token is None:
        phi_tokenizer.pad_token = phi_tokenizer.eos_token

    print("✅ High-quality models loaded successfully!")
    return llama_tokenizer, llama_model, phi_tokenizer, phi_model

# Load the models
llama_tokenizer, llama_model, phi_tokenizer, phi_model = load_models()

In [ ]:
def generate_llama_response(prompt, max_new_tokens=150, temperature=0.7):
    """Generate response using Llama-3.2-3B-Instruct"""

    # Format prompt for Llama instruction format
    formatted_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    inputs = llama_tokenizer(formatted_prompt, return_tensors="pt", truncation=True)
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    with torch.no_grad():
        outputs = llama_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=llama_tokenizer.eos_token_id,
            eos_token_id=llama_tokenizer.eos_token_id,
            use_cache=False
        )

    # Decode only the new tokens (response)
    response = llama_tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
    return response.strip()

def generate_phi_response(prompt, max_new_tokens=150, temperature=0.7):
    """Generate response using Phi-3-mini"""

    # Format prompt for Phi-3 instruction format
    formatted_prompt = f"<|user|>\n{prompt}<|end|>\n<|assistant|>\n"

    inputs = phi_tokenizer(formatted_prompt, return_tensors="pt", truncation=True)
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    with torch.no_grad():
        outputs = phi_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=phi_tokenizer.eos_token_id,
            eos_token_id=phi_tokenizer.eos_token_id,
            use_cache=False  # Disable cache to avoid compatibility issues
        )

    # Decode only the new tokens (response)
    response = phi_tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
    return response.strip()

def display_comparison(title, prompts_and_responses):
    """Display a nice comparison of different prompting approaches"""
    print(f"\n{'='*60}")
    print(f"🎯 {title}")
    print(f"{'='*60}")

    for i, (prompt_type, prompt, response) in enumerate(prompts_and_responses, 1):
        print(f"\n📝 Approach {i}: {prompt_type}")
        print(f"Prompt: {prompt}")
        print(f"Response: {response}")
        print("-" * 40)

print("🛠️ Helper functions ready!")

## 2️⃣ Basic Prompting: The Foundation

Basic prompting is the simplest form of interaction with an LLM. You provide a direct instruction or question, and the model responds. The key is being **clear**, **specific**, and **concise**.

### **🔹 Key Principles of Basic Prompting:**
- **Be Specific**: Vague prompts lead to vague responses
- **Provide Context**: Give the model enough information to understand your request
- **Set Expectations**: Specify the format or type of response you want
- **Use Clear Language**: Avoid ambiguity and complex sentence structures

Let's see how different ways of asking the same question can yield very different results with our high-quality models!

In [ ]:
print("🚀 Basic Prompting Examples")

# Example 1: Math Problem - Different levels of specificity
math_prompts = [
    ("Vague", "Do some math", ""),
    ("Basic", "What is 847 * 23?", ""),
    ("Specific", "Calculate 847 multiplied by 23 and show your work step by step.", ""),
    ("Very Specific", "Solve this multiplication problem step by step: 847 × 23 = ? Show each step of the calculation.", "")
]

# Generate responses for math prompts using Phi-3 (better for math)
for i, (prompt_type, prompt, _) in enumerate(math_prompts):
    if prompt.strip() :
        response = generate_phi_response(prompt)
        math_prompts[i] = (prompt_type, prompt, response)

# Filter out empty responses
math_prompts = [(pt, p, r) for pt, p, r in math_prompts if r]

display_comparison("Basic Prompting: Specificity Matters", math_prompts)

In [ ]:
# Example 2: Creative Writing - Context and Format Matter
writing_prompts = [
    ("No Context", "Write a story.", ""),
    ("Some Context", "Write a short story about a robot discovering emotions.", ""),
    ("Clear Context & Format", "Write a 100-word story about a robot who discovers emotions when it finds an abandoned kitten. Include dialogue and end with a hopeful tone.", "")
]

for i, (prompt_type, prompt, _) in enumerate(writing_prompts):
    response = generate_llama_response(prompt, temperature=0.8)
    writing_prompts[i] = (prompt_type, prompt, response)

display_comparison("Basic Prompting: Context and Format", writing_prompts)

In [ ]:
# Example 3: Information Retrieval - Precision in Questions
info_prompts = [
    ("Broad", "Tell me about climate change.", ""),
    ("Focused", "What are the three main causes of climate change?", ""),
    ("Targeted", "List the top 3 human activities that contribute most to greenhouse gas emissions, with a brief explanation for each.", "")
]

for i, (prompt_type, prompt, _) in enumerate(info_prompts):
    response = generate_llama_response(prompt)
    info_prompts[i] = (prompt_type, prompt, response)

display_comparison("Basic Prompting: Question Precision", info_prompts)

## 3️⃣ Few-shot Learning: Learning by Example

Few-shot learning is like showing someone examples before asking them to do a task. Instead of just giving instructions, we provide the model with **examples** of the input-output pattern we want it to follow.

### **🔹 The Power of Examples:**
- **Pattern Recognition**: The model learns the desired format and style
- **Consistency**: Examples help ensure consistent output structure
- **Complex Tasks**: Enables the model to handle tasks that are hard to describe in words
- **Quality Control**: Examples set the standard for response quality

### **🔹 Few-shot Structure:**
- Example 1: [Input] → [Expected Output]
- Example 2: [Input] → [Expected Output]
- Example 3: [Input] → [Expected Output]
- Now do this: [Your actual input] → [Model generates output]

Our high-quality models excel at pattern recognition, making few-shot learning extremely effective!

In [ ]:
print("🎯 Few-shot Learning Examples")

# Example 1: Sentiment Analysis with Few-shot
few_shot_sentiment_prompt = """
Review: "This restaurant exceeded all my expectations! Amazing food and service."
Sentiment: Positive

Review: "Terrible experience. Cold food, rude staff, overpriced."
Sentiment: Negative

Review: "It was decent. Nothing special but not bad either."
Sentiment: Neutral

Classify the sentiment of the following review as Positive, Negative, or Neutral, using the previous examples:

Review: "The program barely works, but there is no alternative"
Sentiment:"""

# Compare with zero-shot
zero_shot_sentiment_prompt = """
Classify the sentiment of this review as Positive, Negative, or Neutral:

Review: "The program barely works, but there is no alternative"
Sentiment:"""

sentiment_responses = [
    ("Zero-shot", zero_shot_sentiment_prompt, ""),
    ("Few-shot", few_shot_sentiment_prompt, "")
]

for i, (approach, prompt, _) in enumerate(sentiment_responses):
    response = generate_llama_response(prompt, max_new_tokens=30)
    sentiment_responses[i] = (approach, "Software review sentiment", response)

display_comparison("Few-shot vs Zero-shot: Sentiment Analysis", sentiment_responses)

In [ ]:
# Example 2: Code Documentation with Few-shot
few_shot_code_prompt = """
Function: def add_numbers(a, b): return a + b
Documentation: Adds two numbers together and returns the result. Parameters: a (int/float), b (int/float). Returns: sum of a and b.

Function: def find_max(numbers): return max(numbers)
Documentation: Finds the maximum value in a list of numbers. Parameters: numbers (list). Returns: maximum value from the list.

Using the previous two examples as valid documentation, generate documentation for the following function:

Function: def validate_email(email): return "@" in email and "." in email.split("@")[1]
"""

zero_shot_code_prompt = """
Generate documentation for this Python function:

Function: def validate_email(email): return "@" in email and "." in email.split("@")[1]
Documentation:"""

code_responses = [
    ("Zero-shot", zero_shot_code_prompt, ""),
    ("Few-shot", few_shot_code_prompt, "")
]

for i, (approach, prompt, _) in enumerate(code_responses):
    response = generate_phi_response(prompt, max_new_tokens=80)
    code_responses[i] = (approach, "Email validation function", response)

display_comparison("Few-shot vs Zero-shot: Code Documentation", code_responses)

In [ ]:
# Example 3: Complex Pattern - Data Analysis Summary
few_shot_analysis_prompt = """
Dataset: Customer satisfaction survey (n=500, satisfaction score 4.2/5, 85% would recommend)
Summary: HIGH SATISFACTION | Score: 4.2/5 | Recommendation rate: 85% | Sample: 500 customers | Action: Maintain current service quality

Dataset: Website performance metrics (avg load time 2.1s, bounce rate 35%, conversion 3.2%)
Summary: NEEDS IMPROVEMENT | Load time: 2.1s (slow) | Bounce rate: 35% (high) | Conversion: 3.2% | Action: Optimize page speed

Using the previous two datasets as examples, please create a summary for the following dataset, use the same Summary format that was shown above.
Dataset: Sales quarterly report (Q3 revenue $2.1M, 15% growth, target was $2M, top product: Software licenses)
"""

response = generate_llama_response(few_shot_analysis_prompt, max_new_tokens=80)
print(f"\n📊 Few-shot Data Analysis Summary:")
print(f"Input: Sales quarterly report with revenue, growth, and target data")
print(f"Output: {response}")

## 4️⃣ Chain-of-Thought (CoT): Teaching the Model to Think

Chain-of-Thought prompting is like asking someone to "show their work" on a complex problem. Instead of just getting the final answer, we guide the model to **think step-by-step** and show its reasoning process.

### **🔹 Why CoT Works:**
- **Complex Reasoning**: Breaks down difficult problems into manageable steps
- **Improved Accuracy**: Step-by-step thinking reduces errors
- **Transparency**: We can see how the model arrived at its conclusion
- **Debugging**: If the answer is wrong, we can see where the reasoning failed

### **🔹 CoT Structure:**
**Problem:** [Complex question or task]
**Let me think step by step:**

1. [First reasoning step]
2. [Second reasoning step]
3. [Third reasoning step]

...

**Therefore,** [final answer]


Our modern models have excellent reasoning capabilities and respond very well to CoT prompting!

In [ ]:
print("🧠 Chain-of-Thought Reasoning Examples")

# Example 1: Complex Math Word Problem
direct_math_prompt = """
A company's revenue increased by 25% in Year 1, then decreased by 20% in Year 2. If they started with $800,000, what's their final revenue?"""

cot_math_prompt = """
A company's revenue increased by 25% in Year 1, then decreased by 20% in Year 2. If they started with $800,000, what's their final revenue?

Let me solve this step by step:
1. Starting revenue: $800,000
2. Year 1 increase of 25%: $800,000 × 1.25 = $1,000,000
3. Year 2 decrease of 20%: $1,000,000 × 0.80 = $800,000
Therefore, the final revenue is $800,000.

Now solve this problem step by step:
A store's profit margin was 15% in Q1, increased to 22% in Q2, then dropped to 18% in Q3. If Q1 sales were $500,000, and sales increased 10% each quarter, what was the profit in Q3?

Let me solve this step by step:"""

math_responses = [
    ("Direct", "A store's profit margin was 15% in Q1, increased to 22% in Q2, then dropped to 18% in Q3. If Q1 sales were $500,000, and sales increased 10% each quarter, what was the profit in Q3?", ""),
    ("Chain-of-Thought", cot_math_prompt, "")
]

for i, (approach, prompt, _) in enumerate(math_responses):
    response = generate_phi_response(prompt, max_new_tokens=500)
    math_responses[i] = (approach, "Store profit calculation", response)

display_comparison("Chain-of-Thought: Complex Math Problem", math_responses)

In [ ]:
# Example 2: Logical Reasoning
direct_logic_prompt = """
If "No cats are dogs" and "All pets in this house are cats," what can we conclude about dogs in this house?"""

cot_logic_prompt = """
If "No cats are dogs" and "All pets in this house are cats," what can we conclude about dogs in this house?

Let me analyze this step by step:
1. Given: "No cats are dogs" - This means cats and dogs are mutually exclusive categories
2. Given: "All pets in this house are cats" - Every pet in the house belongs to the cat category
3. From (1): If something is a cat, it cannot be a dog
4. From (2): All pets are cats
5. Combining (3) and (4): Since all pets are cats, and no cats are dogs, no pets can be dogs
Therefore, there can be no dogs among the pets in this house.

Now analyze this logic problem step by step:
If "All successful entrepreneurs take risks" and "Maria is risk-averse," what can we conclude about Maria's entrepreneurial success?

Let me analyze this step by step:"""

logic_responses = [
    ("Direct", "If 'All successful entrepreneurs take risks' and 'Maria is risk-averse,' what can we conclude about Maria's entrepreneurial success?", ""),
    ("Chain-of-Thought", cot_logic_prompt, "")
]

for i, (approach, prompt, _) in enumerate(logic_responses):
    response = generate_phi_response(prompt, max_new_tokens=100)
    logic_responses[i] = (approach, "Entrepreneurship logic problem", response)

display_comparison("Chain-of-Thought: Logical Reasoning", logic_responses)

In [ ]:
# Example 3: Complex Decision Making with CoT
decision_prompt = """
Should a small tech startup with 10 employees, $200K runway, and 6 months left accept a $50K investment offer that requires giving up 25% equity?

Let me think through this decision step by step:
1. Current situation analysis:
   - Very limited runway (6 months)
   - Small team size suggests early stage
   - Need capital to survive and grow

2. Investment offer evaluation:
   - $50K extends runway by ~2-3 months (assuming $30K/month burn)
   - 25% equity is significant for a small amount
   - Valuation implied: $200K post-money

3. Alternative considerations:
   - Could they raise more money from other sources?
   - Could they reduce burn rate instead?
   - What's the growth trajectory?

4. Risk assessment:
   - Without funding: likely shutdown in 6 months
   - With funding: more time but significant dilution
   - Investor might provide valuable guidance

Therefore, if no better alternatives exist, accepting the investment is likely the right choice to survive, despite the high dilution.

Now help me think through this decision step by step:
A freelance designer earning $80K/year is offered a full-time position at $70K/year plus benefits (health insurance worth $8K, 401k match worth $3K, paid vacation worth $5K). Should they take it?

Let me think through this decision step by step:"""

response = generate_llama_response(decision_prompt, max_new_tokens=400)
print(f"\n🤔 Chain-of-Thought Decision Making:")
print(f"Problem: Freelancer considering full-time job offer")
print(f"CoT Response: {response}")

## 5️⃣ Zero-shot Chain-of-Thought: The Magic Phrase

Zero-shot Chain-of-Thought is an incredibly simple yet powerful technique. By adding the phrase **"Let's think step by step"** to any prompt, we can often trigger step-by-step reasoning without providing any examples!

### **🔹 The Magic of "Let's think step by step":**
- **No Examples Needed**: Works without providing reasoning examples
- **Universal Trigger**: Works across many different types of problems
- **Emergent Behavior**: The model naturally breaks down complex problems
- **Simple Implementation**: Just add one phrase to your existing prompts

### **🔹 Zero-shot CoT Variations:**
- "Let's think step by step"
- "Let's work through this systematically"
- "Let me break this down step by step"
- "Let's solve this step by step"
- "Think about this carefully and systematically"

Our modern models respond exceptionally well to these reasoning triggers!

In [ ]:
print("✨ Zero-shot Chain-of-Thought: The Magic Phrase")

# Example 1: Complex Calculation - With and Without the Magic Phrase
regular_prompt = "A rectangular garden is 15 meters long and 8 meters wide. If you want to put a fence around it and fence costs $12 per meter, how much will the total cost be?"

zero_shot_cot_prompt = "A rectangular garden is 15 meters long and 8 meters wide. If you want to put a fence around it and fence costs $12 per meter, how much will the total cost be? Let's think step by step."

math_zero_shot = [
    ("Regular Prompt", regular_prompt, ""),
    ("Zero-shot CoT", zero_shot_cot_prompt, "")
]

for i, (approach, prompt, _) in enumerate(math_zero_shot):
    response = generate_phi_response(prompt, max_new_tokens=100)
    math_zero_shot[i] = (approach, "Garden fence problem", response)

display_comparison("Zero-shot CoT: Math Problem", math_zero_shot)

In [ ]:
# Example 2: Strategy Problem
regular_strategy_prompt = "A new social media app has 1000 users after 3 months. Competitors have millions. What should they focus on to grow?"

zero_shot_strategy_prompt = "A new social media app has 1000 users after 3 months. Competitors have millions. What should they focus on to grow? Let's think step by step."

strategy_zero_shot = [
    ("Regular Prompt", regular_strategy_prompt, ""),
    ("Zero-shot CoT", zero_shot_strategy_prompt, "")
]

for i, (approach, prompt, _) in enumerate(strategy_zero_shot):
    response = generate_llama_response(prompt, max_new_tokens=200)
    strategy_zero_shot[i] = (approach, "App growth strategy", response)

display_comparison("Zero-shot CoT: Strategy Problem", strategy_zero_shot)


In [ ]:

# Example 3: Technical Debugging
regular_debug_prompt = "My Python web app is running slowly. Users complain about 5-second load times. How should I troubleshoot this?"

zero_shot_debug_prompt = "My Python web app is running slowly. Users complain about 5-second load times. How should I troubleshoot this? Let's think step by step."

debug_zero_shot = [
    ("Regular Prompt", regular_debug_prompt, ""),
    ("Zero-shot CoT", zero_shot_debug_prompt, "")
]

for i, (approach, prompt, _) in enumerate(debug_zero_shot):
    response = generate_phi_response(prompt, max_new_tokens=300)
    debug_zero_shot[i] = (approach, "Performance debugging", response)

display_comparison("Zero-shot CoT: Technical Problem", debug_zero_shot)

## 6️⃣ Advanced Prompt Engineering Techniques

Now that we've mastered the basics, let's explore some advanced techniques that can further improve your prompt engineering skills:

### **🔹 Role-Playing**: Having the AI adopt specific personas or expertise
### **🔹 Output Formatting**: Controlling the structure and format of responses
### **🔹 Constraint Setting**: Adding specific limitations or requirements
### **🔹 Multi-step Prompting**: Breaking complex tasks into smaller steps
### **🔹 Self-Correction**: Having the model check and improve its own work

In [ ]:
print("🚀 Advanced Prompt Engineering Techniques")

# Technique 1: Expert Role-Playing
role_playing_prompt = """
You are a senior cybersecurity expert with 15 years of experience in enterprise security.
A small business owner asks: "I have 20 employees using personal devices for work. What are the top 3 security risks I should address immediately?"

Provide specific, actionable advice with your expert perspective:"""

response = generate_llama_response(role_playing_prompt, max_new_tokens=400)
print(f"\n💼 Expert Role-Playing Technique:")
print(f"Response: {response}")


In [ ]:

# Technique 2: Structured Output Formatting
formatting_prompt = """
Analyze the pros and cons of electric vehicles vs gasoline cars. Format your response exactly as:

ELECTRIC VEHICLES:
Advantages:
- [advantage 1 with brief explanation]
- [advantage 2 with brief explanation]
- [advantage 3 with brief explanation]
Disadvantages:
- [disadvantage 1 with brief explanation]
- [disadvantage 2 with brief explanation]

GASOLINE CARS:
Advantages:
- [advantage 1 with brief explanation]
- [advantage 2 with brief explanation]
- [advantage 3 with brief explanation]
Disadvantages:
- [disadvantage 1 with brief explanation]
- [disadvantage 2 with brief explanation]"""

response = generate_llama_response(formatting_prompt, max_new_tokens=500)
print(f"\n📋 Structured Output Formatting:")
print(f"Response: {response}")


In [ ]:

# Technique 3: Self-Correction
self_correction_prompt = """
Solve this problem: "If a train travels 240 km in 3 hours, what's its average speed in mph?"

First, provide your initial answer. Then, check your work and identify any errors. Finally, provide the corrected answer if needed.

Initial solution:"""

response = generate_phi_response(self_correction_prompt, max_new_tokens=1000)
print(f"\n🔍 Self-Correction Technique:")
print(f"Response: {response}")

In [ ]:
# Technique 4: Multi-step Complex Problem Solving
complex_prompt = """
You're helping a startup founder make a critical decision. They have these options:

Option A: Raise $500K VC funding (30% equity, 18 months runway)
Option B: Take $100K angel investment (8% equity, 6 months runway)
Option C: Bootstrap with current $50K savings (3 months runway)

Additional context: B2B SaaS product, 2 co-founders, early traction (10 paying customers, $2K MRR), growing 20% monthly.

Please analyze this systematically:

Step 1: Evaluate each option's financial implications
Step 2: Assess the strategic value and risks
Step 3: Consider timeline and growth requirements
Step 4: Make a recommendation with reasoning

Step 1 - Financial Analysis:"""

response = generate_llama_response(complex_prompt, max_new_tokens=1500)
print(f"\n🎯 Multi-step Complex Problem Solving:")
print(f"Response: {response}")

## 7️⃣ Prompt Engineering Best Practices & Tips

### **🎯 The Golden Rules of Prompt Engineering:**

#### **1. Clarity is King**
- Use simple, clear language
- Avoid ambiguous terms
- Be specific about what you want

#### **2. Context is Crucial**
- Provide relevant background information
- Set the scene for your request
- Include constraints and requirements
- How to get the context you may ask?
![RAGS](https://m.media-amazon.com/images/I/91k5FXf3d6L.jpg)

#### **3. Examples Are Powerful**
- Show, don't just tell
- Use diverse examples
- Include edge cases when relevant

#### **4. Structure Matters**
- Use clear formatting and organization
- Break complex tasks into steps
- Specify desired output format

#### **5. Iterate and Refine**
- Start simple, then add complexity
- Test variations of your prompts
- Learn from what works and what doesn't

#### **6. Know Your Model**
- Modern models like Llama-3.2 and Phi-3 are highly capable
- They understand context and nuance well
- They respond excellently to reasoning prompts

### **⚡ Pro Tips for Better Results:**
- Use "Let's think step by step" for complex problems
- Add "You are an expert in..." for specialized knowledge
- Include "Be specific and detailed" for comprehensive answers
- Use "Format your response as..." for structured output
- Try "First..., then..., finally..." for multi-step tasks

### **🚫 Common Mistakes to Avoid:**
- Being too vague in your requests
- Not providing enough context
- Asking multiple unrelated questions in one prompt
- Forgetting to specify the desired output format
- Not testing your prompts with variations

In [ ]:
print("🎓 Practical Exercise: Design Your Own Prompts")

# Exercise: Create different prompts for the same task
task = "Help someone create a comprehensive business plan for a food truck"
prompts_to_test = [


]
# Design prompts using different techniques!
# Example prompts:
# prompts_to_test = [
#     ("Basic", "How do I write a business plan for a food truck?"),
#
#     ("Specific + Context", "I want to start a gourmet burger food truck in Austin, Texas with $80,000 startup capital. Create a comprehensive business plan covering market analysis, financial projections, and operations."),
#
#     ("Expert Role-playing", "You are a successful food truck entrepreneur and business consultant with 10 years of experience. Help me create a detailed business plan for a gourmet burger food truck in Austin, Texas with $80,000 startup capital. Include specific insights from your experience."),
#
#     ("Zero-shot CoT", "I need a business plan for a gourmet burger food truck in Austin, Texas with $80,000 startup capital. Let's think step by step about all the components I need to address."),
#      ]

print("\n🔍 Testing Different Prompt Approaches for Business Plan:")
for approach, prompt in prompts_to_test:
    response = generate_llama_response(prompt, max_new_tokens=150)
    print(f"\n📝 {approach}:")
    print(f"Response: {response}")
    print("-" * 50)

#In conclusion

![Prompting](https://i.imgur.com/iaEypZu.png)

### Contributed by: Ali Habibullah

